In [2]:
import os
import shutil
import boto3
import tempfile

# Access the S3 bucket
AWS_ACCESS_KEY_ID = "aws_access_key_id"
AWS_SECRET_ACCESS_KEY = "aws_secret_access_key"
# aws_session_token : token is needed only when you approach once
AWS_BUCKET_NAME = "my_bucket" # the part after s3:// > s3://my_bucket 

session = boto3.session.Session(aws_access_key_id=AWS_ACCESS_KEY_ID,
                               aws_secret_access_key=AWS_ACCESS_KEY_ID)
s3 = session.resource("s3")


def save_file_to_s3(target, aws_key:str, file_name:str, is_dir=False):
    """
    Function to save a file/a directory to AWS s3 bucket.
    You need to figure out the type of target file and address it.

    target: a file/a directory you want to upload
    aws_key: path of the directory of s3 bucket that you want to save the targer file in
    file_name: a name of target you want to save as
    is_dir: if True, it needs to be zipped.
    """
    if is_dir:
        # the entire model, tfrecords etc.
        with tempfile.TemporaryDirectory() as tempdir:
            TEMP_DIR = os.path.join(tempdir, file_name)
            target.save(TEMP_DIR)
            shutil.make_archive(f"{TEMP_DIR}.zip", "zip", TEMP_DIR)
            s3.meta.client.upload_file(f"{TEMP_DIR}.zip",
                                        AWS_BUCKET_NAME,
                                        f"{aws_key}/{file_name}.zip")
    
    else:
        # xlsx, csv etc.
        key = f"{aws_key}/{file_name}.csv"  # change the format if needed.
        upload = s3.Object(AWS_BUCKET_NAME, key)
        upload.put(Body=target.to_csv(None, index=False)) # change the format if needed.


def load_file_from_s3(aws_key:str, file_name:str):
    """
    Function to download a file saved to AWS s3 bucket.
    You can only load a file with this function(not a directory).

    aws_key: path of the directory of s3 bucket that contains the file you want to download
    file_name: the name of the file
    """
    key = f"{aws_key}/{file_name}.csv" # change the format if needed.
    obj = s3.Object(AWS_BUCKET_NAME, key).get()
    return pd.read_csv(obj['Body'])

# If you want to download a zip file and load model or tfrecord datasets directly,
# you can user tempfile.TemporaryDirectory like when saving it.
# You also need to use shutil.unpack_archive to unzip the zip file and load it.